# Alignment tolerance

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import astropy.units as u

%matplotlib inline

In [ ]:
datpath = '/melkor/d1/guenther/processing/redsox/'

In [ ]:
import sys
sys.path.append('..')
from redsox.mirror import Ageom

In [ ]:
from marxs.design.tolerancing import select_1dof_changed

def plot_wiggle(tab, par, parlist, ax, axt=None,
                modfac='mod_mean', Aeff_col='Aeff_channel',
                axes_facecolor='w', MDP=False):
    '''Plotting function for overview plot wiggeling 1 dof at the time.

    For parameters starting with "d" (e.g. "dx", "dy", "dz"), the plot axes
    will be labeled as a shift, for parameters tarting with "r" as rotation.

    Parameters
    ----------
    table : `astropy.table.Table`
        Table with wiggle results
    par : string
        Name of parameter to be plotted
    parlist : list of strings
        Name of all parameters in ``table``
    ax : `matplotlib.axes.Axes`
        Axis object to plot into.
    axt : ``None`` or  `matplotlib.axes.Axes`
        If this is ``None``, twin axis are created to show resolving power
        and effective area in one plot. Alternatively, a second axes instance
        can be given here.
    R_col : string
        Column name in ``tab`` that hold the resolving power to be plotted.
        Default is set to work with `marxs.design.tolerancing.CaptureResAeff`.
    Aeff_col : string
        Column name in ``tab`` that hold the effective area to be plotted.
        Default is set to work with `marxs.design.tolerancing.CaptureResAeff`.
    axes_facecolor : any matplotlib color specification
        Color for the background in the plot.
    '''
    import matplotlib.pyplot as plt

    t = select_1dof_changed(tab, par, parlist)
    t.sort(par)
    t_wave = t.group_by('wave')
    if (axt is None) and not MDP:
        axt = ax.twinx()

    for key, g in zip(t_wave.groups.keys, t_wave.groups):
        if par[0] == 'd':
            x = g[par]
        elif par[0] == 'r':
            x = np.rad2deg(g[par].data)
        else:
            raise ValueError("Don't know how to plot {}. Parameter names should start with 'd' for shifts and 'r' for rotations.".format(par))

        if MDP:
            ax.plot(x, np.abs(g['modulation'][:, 1]) * np.sqrt(g[Aeff_col]), 
                    label='{:3.1f} $\AA$'.format(key[0]), lw=2 )
        else:
            ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
            axt.plot(x, Ageom.to(u.cm**2) * g[Aeff_col], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    if MDP:
        ax.set_ylabel('Figure of merrit')
        axlist = [ax]
    else:
        ax.set_ylabel('Modulation factor (solid lines)')
        axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
        axlist = [ax, axt]
        
    if par[0] == 'd':
        ax.set_xlabel('shift [mm]')
        ax.set_title('Shift along {}'.format(par[1]))
    elif par[0] == 'r':
        ax.set_xlabel('Rotation [degree]')
        ax.set_title('Rotation around {}'.format(par[1]))

    for a in axlist:
        a.set_facecolor(axes_facecolor)
        a.set_axisbelow(True)
        a.grid(axis='x', c='1.0', lw=2, ls='solid')



wiggle_plot_facecolors = {'global': '0.9',
                          'individual': (1.0, 0.9, 0.9)}
'''Default background colors for wiggle overview plots.

If the key of the dict matches part of the filename, the color listed in
the dict is applied.
'''

def load_and_plot(filename, parlist=['dx', 'dy', 'dz', 'rx', 'ry', 'rz'], **kwargs):
    '''Load a table with wiggle results and make default plot

    This is a function to generate a quicklook image with many
    hardcoded defaults for figure size, colors etc.
    In particular, this function is written for the display of
    6d plots which vary 6 degrees of freedom, one at a time.

    The color for the background in the plot is set depending on the filename
    using the ``string : color`` assignments in
    `~marxs.design.tolerancing.wiggle_plot_facecolors`. No fancy regexp based
    match is applied, this is simply a check with ``in``.

    Parameters
    ----------
    filename : string
        Path to a file with data that can be plotted by
        `~marxs.design.tolerancing.plot_wiggle`.

    parlist : list of strings
        Name of all parameters in ``table``.
        This function only plots six of them.

    Returns
    -------
    tab : `astropy.table.Table`
        Table of data read from ``filename``
    fig : `matplotlib.figure.Figure`
        Figure with plot.
    kwargs :
        All other parameters are passed to
        `~marxs.design.tolerancing.plot_wiggle`.
    '''
    import matplotlib.pyplot as plt

    tab = Table.read(filename)

    if 'axis_facecolor' not in kwargs:
        for n, c in wiggle_plot_facecolors.items():
            if n in filename:
                kwargs['axes_facecolor'] = c

    fig = plt.figure(figsize=(12, 8))
    fig.subplots_adjust(wspace=.6, hspace=.3)
    for i, par in enumerate(parlist):
        ax = fig.add_subplot(2, 3, i + 1)
        plot_wiggle(tab, par, parlist, ax, **kwargs)

    return tab, fig

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'mirror_global.fits'))

In [ ]:
fig, ax = plt.subplots()

t = select_1dof_changed(tab, 'dy', ['dx', 'dy', 'dz', 'rx', 'ry', 'rz'])
t.sort('dy')
t_wave = t.group_by('wave')

for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['dy']
    for i in [1]:
        ax.plot(x, g['Aeff'][:, i], label=i, lw=2)
ax.legend()

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_global.fits'))

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_individual.fits'))

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'mlmirror_global.fits'))

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'detector_global.fits'))

In [ ]:
t = Table.read(os.path.join(datpath, 'CAT_period.fits'))
fig = plt.figure()
ax = fig.add_subplot(111)
t_wave = t.group_by('wave')
axt = ax.twinx()
    
for key, g in zip(t_wave.groups.keys, t_wave.groups):
    x = g['period_sigma'] / g['period_mean']
    ax.set_xlabel('relative change in grating period')
    
    ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
    axt.plot(x, Ageom.to(u.cm**2) * g['Aeff_channel'], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
    ax.set_ylabel('Modulation factor (solid lines)')
    axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')


ax.set_facecolor((0.9, 0.9, 1.))
ax.set_axisbelow(True)
ax.grid(axis='x', c='1.0', lw=2, ls='solid')
ax.legend()
ax.set_xscale("log")
ax.set_title('Variation of the grating period')
out = ax.set_xlim([np.min(x), None])

In [ ]:
tab = Table.read(os.path.join(datpath, 'scatter.fits'))
fig = plt.figure(figsize=(12, 5))

for i, par in enumerate(['inplanescatter', 'perpplanescatter']):
    t = select_1dof_changed(tab, par, ['inplanescatter', 'perpplanescatter'])
    ax = fig.add_subplot(1, 2, i+1)
    t_wave = t.group_by('wave')
    axt = ax.twinx()
    
    for key, g in zip(t_wave.groups.keys, t_wave.groups):
        x = np.rad2deg(g[par])*60
        ax.set_title(par)
        ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
        axt.plot(x, Ageom.to(u.cm**2) * g['Aeff_channel'], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
        ax.set_ylabel('Modulation factor (solid lines)')
        axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
        ax.set_xlabel('Gaussian $\sigma$ in arcmin')
    ax.set_facecolor((0.9, 0.9, 1.))
    ax.set_axisbelow(True)
    ax.grid(axis='x', c='1.0', lw=2, ls='solid')
    ax.legend()
    out = ax.set_xlim([np.min(x), None])

In [ ]:
tab = Table.read(os.path.join(datpath, 'offset_point.fits'))
fig = plt.figure(figsize=(12, 5))

for i, par in enumerate(list(set(tab['position_angle']))):
    t = tab[tab['position_angle'] == par]
    ax = fig.add_subplot(1, 2, i+1)
    t_wave = t.group_by('wave')
    axt = ax.twinx()
    
    for key, g in zip(t_wave.groups.keys, t_wave.groups):
        x = g['separation']
        ax.set_title('offset angle {}'.format(par))
        ax.plot(x, np.abs(g['modulation'][:, 1]), label='{:3.1f} $\AA$'.format(key[0]), lw=1.5)
        axt.plot(x, Ageom.to(u.cm**2) * g['Aeff_channel'], ':', label='{:2.0f} $\AA$'.format(key[0]), lw=2)
        ax.set_ylabel('Modulation factor (solid lines)')
        axt.set_ylabel('$A_{eff}$ [cm$^2$] per channel (dotted lines)')
        ax.set_xlabel('offset in arcsec')
    ax.set_facecolor((0.9, 0.9, 1.))
    ax.set_axisbelow(True)
    ax.grid(axis='x', c='1.0', lw=2, ls='solid')
    ax.legend()
    out = ax.set_xlim([np.min(x), None])

## Repeat plots with FOM

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'mirror_global.fits'), MDP=True)
fig.axes[0].set_xlim([-2, 2])
fig.axes[1].set_xlim([-2, 2])

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_global.fits'), MDP=True)
fig.axes[3].set_xlim([-1, 1])
fig.axes[4].set_xlim([-1, 1])

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'CAT_individual.fits'), MDP=True)

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'mlmirror_global.fits'), MDP=True)
fig.axes[0].set_xlim([-1, 1])
fig.axes[1].set_xlim([-1, 1])
fig.axes[3].set_xlim([-1, 1])
fig.axes[4].set_xlim([-1, 1])

In [ ]:
tab, fig = load_and_plot(os.path.join(datpath, 'detector_global.fits'), MDP=True)

## Step 2

Here is the alignment table. The numbers in that table are tighter than they should be in ML mirror trans y because the misalignment are applied in the global coordinate system and thus +x means something different for each mirror. I looked at the "per channel" results to figure this out, and it would be better to apply misalignments in local coordinates where +x means "shift along the axis where the period changes" as opposed to shift in the global coordinate system. Since two mirrors are involved in the calculation of the MPF, that means both x and y look tight, when really it's tight only along the direction in which the mirror is graded. Until MARXS offers that capability, this needs to be hand-checked.

In [ ]:
from redsox.redsox import align_requirement_moritz as arm
from astropy.table import QTable
from astropy.units import Quantity

talign = QTable([[a[4] for a in arm]], names=['alignment'])
for i, col in enumerate(['trans x', 'trans y', 'trans z']):
    talign[col] = Quantity([a[2][i].to(u.mm) for a in arm])
    talign[col].format = '{:5.1f}'
    
for i, col in enumerate(['rot x', 'rot y', 'rot z']):
    talign[col] = Quantity([a[2][i + 3].to(u.arcmin) for a in arm])
    talign[col].format = '{:5.0f}'
talign

In [ ]:
talign.write(sys.stdout, format='ascii.latex')

In [ ]:
tbase = Table.read(os.path.join(datpath, 'moritz_budget.fits'))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

waves = list(set(tbase['wave']))
waves.sort()

tbase['FOM'] = np.abs(tbase['modulation'][:, 1]) * np.sqrt(tbase['Aeff_channel'])

for i, wave in enumerate(waves):
    tw = tbase[tbase['wave'] == wave]
    ax.hist(tw['FOM'][1:] / tw['FOM'][0], label='${:2.0f}\;\AA$'.format(wave), 
            histtype='stepfilled',
            alpha=0.6, bins=np.arange(.5, 1.051, .05))
    
ax.set_xlabel('FOM relative to perfect alignment')
ax.set_ylabel('Simulations')

out = ax.legend()

#fig.savefig(os.path.join(get_path('figures'), 'alignbudget.pdf'), bbox_inches='tight') 

In [ ]:
# Weight each performance with the relative number of counts expected at that wavelength.
# It would be better to run the 100 simulations with the real input spectrum and add up, but this will do for now.
t40 = tbase[tbase['wave'] == 40]
t55 = tbase[tbase['wave'] == 55]
t70 = tbase[tbase['wave'] == 70]
FOM = 18 * t40['FOM'] + 40 * t55['FOM'] + 10 * t70['FOM']


In [ ]:
fig, ax = plt.subplots()
ax.hist(FOM[1:] / FOM[0], label='${:2.0f}\;\AA$'.format(wave), histtype='stepfilled',
            bins=np.arange(.55, 1.051, .05))
    
ax.set_xlabel('FOM relative to perfect alignment')
ax.set_ylabel('Simulations')
fig.savefig('../prop_plots/alignment.pdf', bbox_inches='tight')

In [ ]:
pwd

In [ ]:
fig, ax = plt.subplots()
ax.hist(FOM[1:] / FOM[0], label='${:2.0f}\;\AA$'.format(wave), histtype='stepfilled',
            bins=np.arange(.55, 1.051, .01), cumulative=True)
    
ax.set_xlabel('FOM relative to perfect alignment')
ax.set_ylabel('Simulations')
#fig.savefig('../prop_plots/alignment_cumulative.pdf', bbox_inches='tight')